<a href="https://colab.research.google.com/github/antnewman/nlp-infoextract-notebook/blob/main/nlp_infoextract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Extraction Notebook

**Information Extraction**

There is a plethora of information obtained within text data. Usually, some will be relevant and some irrelevant. Sometimes one might want to extract the names of entities, other times the relationships between certain entities.

# Setup

Check Python version and install appropriate minicoda.

In [ ]:
!which python # should return /usr/local/bin/python
!python --version

In [ ]:
!echo $PYTHONPATH # If /env/python then unset the path, becaue this directory doesn't seeem to exist within the Google Colab file system 

Unset pythonpath variable before installing Miniconda as it can cause problems if there are packages installed and accessible via directories included in the PYTHONPATH that are not compatible with the version of Python included with Miniconda.

In [ ]:
%env PYTHONPATH=

**Installing Miniconda**

Download the installer script for the appropriate version of Miniconda and install it into /usr/local. 

Installing directly into /usr/local, rather than into the default location ~/miniconda3, insures that Conda and all its required dependencies will be automatically available for use within Google Colab.

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Verify that:
- the conda executable is available
- the version is correct
- Installing has not impacted the python executable
- Verify which version of Python has been install by Miniconda

In [ ]:
!which conda # should return /usr/local/bin/conda
!conda --version # should return 4.5.4
!which python # still returns /usr/local/bin/python
!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

**Updating Conda**

We need to update Conda and its dependencies to their most recent versions without updating Python beyond 3.7.

In [ ]:
%%bash

# Updates Conda to the most recent version, but hold Python version fixed at 3.7
conda install --channel defaults conda python=3.7 --yes

# Updates all of Conda’s dependencies to their most recent versions.
conda update --channel defaults --all --yes 

Check versions of conda and python.

In [ ]:
!conda --version # now returns 4.9.2
!python --version # now returns Python 3.7.9 :: Anaconda, Inc.

**Append to** *sys.path* 

We need to add the directory, to which Conda will install packages to the list of directories that Python will search when looking for modules to import.

Check the current list of dirs that Python will search by inspecting the *sys.path*.

In [ ]:
import sys
sys.path

Pre-installed packages are in dist-packages. Conda installed packages are in site-packages.

In [ ]:
!ls /usr/local/lib/python3.7/dist-packages

In [ ]:
import sys
_ = (sys.path
from py2neo import Graph        .append("/usr/local/lib/python3.7/site-packages"))

Note that the dist-packages directory containing the pre-installed Colab packages appears ahead of the site-packages directory where Conda installs packages, henceforth the version of a package available via Colab will take precedence over any version of the same package installed via Conda.

**Installing packages**

Remember to include the --yes flag when installing packages to avoid getting prompted to confirm the package plan.

In [ ]:
!conda install --channel conda-forge featuretools --yes
!conda install -c conda-forge py2neo --yes
!conda install -c conda-forge neuralcoref --yes

**Import**

In [ ]:
from py2neo import Graph

import spacy
import urllib
import neuralcoref
import itertools
import json
from string import punctuation
import nltk

from string import punctuation
from flask import Flask, request
from string import punctuation


**Connect to Neo4J Sandbox**

In [ ]:
# Change the line of code below to use your Bolt URL, and Password of your Sandbox.
# graph = Graph("<Bolt URL>", auth=("neo4j", "<Password>"))
graph = Graph("bolt://3.84.29.113:7687", auth=("neo4j", "distortions-capability-flower"))

**Set Entity Types**

In [ ]:
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]